In [3]:
# !pip install pingouin

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from pingouin import partial_corr

# Load the data
data = pd.read_csv('Train/Compiled.csv', parse_dates=['datetime'])

# Create lagged features for load
num_lags = 168  # Number of past lags
for i in range(1, num_lags + 1):
    data[f'load_lag_{i}'] = data['load'].shift(i)

# Drop rows with missing values due to lagging
data.dropna(inplace=True)

# Calculate partial correlations between load and lagged load variables
lagged_columns = [f'load_lag_{i}' for i in range(1, num_lags + 1)]
partial_correlations = {}
for lag_column in lagged_columns:
    other_columns = [col for col in lagged_columns if col != lag_column]
    data_temp = data[[lag_column, 'load'] + other_columns]
    partial_corr_result = partial_corr(data_temp, x=lag_column, y='load', covar=other_columns, method='pearson')
    partial_correlation = partial_corr_result['r'][0]
    partial_correlations[lag_column] = partial_correlation

# Select lags with partial correlations above a certain threshold
threshold = 0.05  # You can adjust this threshold
selected_lags = [lag for lag, corr in partial_correlations.items() if abs(corr) > threshold]




# Split data into features (exogenous variables and selected lagged load) and target (load)
exogenous_columns = ['pressure_f', 'cloud_cov_f', 'temp_f', 'wind_dir_f', 'wind_sp_f']
features = data[lagged_columns + exogenous_columns]
target = data['load']

# Split data into training and testing sets

train_size = 29232
train_features, test_features = features[:train_size], features[train_size:]
train_target, test_target = target[:train_size], target[train_size:]



# Train a linear regression model
model = LinearRegression()
model.fit(train_features, train_target)

# Initialize an array to store predictions
forecast_horizon = 48
num_forecasts = len(test_target) // 24 - 1
forecasted_values = np.zeros((num_forecasts, forecast_horizon))




# Recursive multi-step forecasting
for i in range(num_forecasts):
    start_index = i * 24
    forecast_features = test_features.iloc[start_index : start_index + 1]
    forecast = model.predict(forecast_features[selected_lags + exogenous_columns])

    # Store the first forecasted value
    forecasted_values[i, 0] = forecast

    # Update the feature matrix for the next prediction
    for lag in range(168,1,-1):
        forecast_features[f'load_lag_{lag}'] = forecast_features[f'load_lag_{lag-1}']
    forecast_features['load_lag_1'] = forecast

    # Perform recursive forecasting for the remaining steps
    for j in range(1, forecast_horizon):
        forecast = model.predict(forecast_features[selected_lags + exogenous_columns])
        forecasted_values[i, j] = forecast

        # Update the feature matrix for the next prediction
        for lag in range(168,1,-1):
            forecast_features[f'load_lag_{lag}'] = forecast_features[f'load_lag_{lag-1}']
        forecast_features['load_lag_1'] = forecast

        
        
forecasted_values = pd.DataFrame(data=forecasted_values)

test_target_array = np.zeros((num_forecasts, forecast_horizon))
for i in range(num_forecasts):
    for j in range(forecast_horizon):
        test_target_array[i,j] = test_target[29400+24*i+j]
test_target_array = pd.DataFrame(data=test_target_array)



error_measures = {}
for i in range(len(forecasted_values.columns)):
    col_name = forecasted_values.columns[i]
    
    mae = mean_absolute_error(test_target_array[col_name], forecasted_values[col_name])
    mse = mean_squared_error(test_target_array[col_name], forecasted_values[col_name])
    rmse = np.sqrt(mse)
    
    absolute_error = np.abs(test_target_array[col_name] - forecasted_values[col_name])
    mape = np.mean(absolute_error / test_target_array[col_name]) * 100
    smape = np.mean(2 * absolute_error / (test_target_array[col_name] + forecasted_values[col_name])) * 100
    
    mase_numerator = np.mean(absolute_error)
    mase_denominator = np.mean(np.abs(test_target_array[col_name].diff()))
    mase = mase_numerator / mase_denominator
    
    error_measures[col_name] = [mae, mse, rmse, mape, smape, mase]

# Create a DataFrame to display error measures
error_df = pd.DataFrame.from_dict(error_measures, orient='index',
                                  columns=['MAE', 'MSE', 'RMSE', 'MAPE', 'SMAPE', 'MASE'])

error_df

for error_measure in error_df.columns:
    plt.figure(figsize=(12, 8))
    plt.bar(error_df.index, error_df[error_measure])
    plt.title(f'{error_measure} for Forecasted Values vs. Actual Values')
    plt.xlabel('Columns')
    plt.ylabel('Error Value')
    plt.xticks(rotation=45)
    plt.show()

    
plt.figure(figsize=(12, 6))
plt.plot(range(0,48), forecasted_values.iloc[1,].values, label='Predicted', linestyle='dashed')
plt.plot(range(0,48), test_target_array.iloc[1,].values, label='Actual')
plt.xlabel('Datetime')
plt.ylabel('Load')
plt.title('Actual Load vs Predicted Load (Linear Regression) First 24 Hours')
plt.legend()
plt.grid()
plt.show()

/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_53150/1720740114.py:9: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data = pd.read_csv('Train/Compiled.csv', parse_dates=['datetime'])
/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_53150/1720740114.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'load_lag_{i}'] = data['load'].shift(i)
/var/folders/34/n9z2vp496dx5gwr2m5_w4p780000gn/T/ipykernel_53150/1720740114.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- load_lag_10
- load_lag_100
- load_lag_101
- load_lag_102
- load_lag_103
- ...
